[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/map_pdfm_features.ipynb)

**Mapping PDFM Features and Predicted Housing Prices**

## Useful Resources

- [Google's Population Dynamics Foundation Model (PDFM)](https://github.com/google-research/population-dynamics)
- Request access to PDFM embeddings [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings)
- Zillow data can be accessed [here](https://www.zillow.com/research/data/)

## Installation

Uncomment and run the following cell to install the required libraries.

In [ ]:
# %pip install "leafmap[maplibre]" scikit-learn

## Import Libraries

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file
import leafmap.maplibregl as leafmap

## Download Zillow Data

Download the Zillow home value data at the county level.

In [ ]:
zhvi_url = "https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_county.csv"
zhvi_file = "data/zillow_home_value_index_by_county.csv"

In [ ]:
if not os.path.exists(zhvi_file):
    download_file(zhvi_url, zhvi_file)

## Process Zillow Data

In [ ]:
zhvi_df = pd.read_csv(
    zhvi_file, dtype={"StateCodeFIPS": "string", "MunicipalCodeFIPS": "string"}
)
zhvi_df.index = "geoId/" + zhvi_df["StateCodeFIPS"] + zhvi_df["MunicipalCodeFIPS"]
zhvi_df.head()

## Request access to PDFM Embeddings

To request access to PDFM embeddings, please follow the instructions [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings).

In [ ]:
county_geojson = "data/county.geojson"
if not os.path.exists(county_geojson):
    raise FileNotFoundError("Please request the embeddings from Google")

## Load county boundaries

In [ ]:
county_gdf = gpd.read_file(county_geojson)
county_gdf.set_index("place", inplace=True)
county_gdf.head()

## Join home value data and county boundaries

In [ ]:
df = zhvi_df.join(county_gdf)
zhvi_gdf = gpd.GeoDataFrame(df, geometry="geometry")
zhvi_gdf.head()

In [ ]:
column = "2024-10-31"
gdf = zhvi_gdf[["RegionName", "State", column, "geometry"]]
gdf.head()

## Visualize home values in 2D

In [ ]:
m = leafmap.Map(style="liberty")
first_symbol_id = m.find_first_symbol_layer()["id"]
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title="Median Home Value",
    name="Median Home Value",
    before_id=first_symbol_id,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/df616803-87e9-4326-be8c-9de5a72a2d95)

## Visualize home values in 3D

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title="Median Home Value",
    extrude=True,
    scale_factor=3,
    before_id=first_symbol_id,
    name="Median Home Value",
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/65f085bb-b781-41b8-8408-440ed202b766)

## Load PDFM county embeddings

In [ ]:
embeddings_file_path = "data/county_embeddings.csv"

In [ ]:
embeddings_df = pd.read_csv(embeddings_file_path).set_index("place")
embeddings_df.head()

In [ ]:
df = embeddings_df.join(county_gdf)
embeddings_gdf = gpd.GeoDataFrame(df, geometry="geometry")
embeddings_gdf.head()

## Visualize PDFM features

Select any of the 329 PDFM features to visualize.

In [ ]:
column = "feature329"  # Change this to the feature you want to use
gdf = embeddings_gdf[[column, "state", "county", "geometry"]]
gdf.head()

In [ ]:
m = leafmap.Map(style="liberty")
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title=column,
    before_id=first_symbol_id,
    name=column,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/efa4983d-89d2-4dcc-9cd6-fefeee069bf7)

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title=column,
    before_id=first_symbol_id,
    name=column,
    extrude=True,
    scale_factor=0.00005,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/fd1b233b-2be0-47c3-b2d0-f53601020604)

## Join Zillow and PDFM Data

In [ ]:
data = zhvi_df.join(embeddings_df, how="inner")
data.head()

In [ ]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "2024-10-31"  # Change this to the date you want to predict

In [ ]:
data = data.dropna(subset=[label])

## Split Train and Test Data

In [ ]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Fit Linear Regression Model

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Evaluate Linear Regression Model

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

In [ ]:
xy_lim = (0, 1_000_000)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

![image](https://github.com/user-attachments/assets/a16a15c6-508b-40c9-aa57-8e98b8b8e216)

## Join predicted values with county boundaries

In [ ]:
df = evaluation_df.join(gdf)
df["difference"] = df["y_pred"] - df["y"]
evaluation_gdf = gpd.GeoDataFrame(df, geometry="geometry")
evaluation_gdf.drop(columns=["category", "color", column], inplace=True)
evaluation_gdf.head()

## Visualize actual home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="Blues",
    column="y",
    legend_title="Actual Home Value",
    before_id=first_symbol_id,
    name="Actual Home Value",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/a86401d5-defd-4277-877b-c8f8c5e07651)

## Visualize predicted home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="Blues",
    column="y_pred",
    legend_title="Predicted Home Value",
    before_id=first_symbol_id,
    name="Predicted Home Value",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/6bfe1c89-968e-4eef-84cc-98bcf07c8acb)

## Visualize difference between predicted and actual home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="coolwarm",
    column="difference",
    legend_title="y_pred-y",
    before_id=first_symbol_id,
    name="Difference",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/4ae58eb1-ab28-49e5-b352-d769a32d3e5c)